In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS
import csv
import json
import time
import pandas as pd

url = 'https://betscsgo.gg/history/'
driver = webdriver.Chrome('C:\Projects_Learnpython\chromedriver')
driver.get(url)

In [2]:
# Парсинг со всех страничек сайта истории

left_team_money_all = [] # Список с деньгами, поставленными на левую команду
right_team_money_all = [] # Список с деньгами, поставленными на правую команду
winning_team_past = [] # Список с победителями прошедших матчпей
dictionary_past_results = [] # Список словарей с результатами работы программы
hyps = [] # Список гипотез

for page_number in range(1, 99):
    url = 'https://betscsgo.gg/history/{}/'.format(page_number)
    driver.get(url)
    time.sleep(15)
    soup = BS(driver.page_source, 'html.parser')
    left_teams = soup.findAll("div", class_="bet-team__name sys-t1name") # Название команды, написанной слева
    right_teams = soup.findAll("div", class_="bet-team__name sys-t2name") # Название команды, написанной справа
    left_team_money = soup.findAll("div",\
    class_ = "bet-button__content-main bet-currency bet-currency_RUB sys-stat-abs-1") # Количество поставленных денег на первую команду
    for lmoney in left_team_money: # Проверка полной длины списка
        left_team_money_all.append(lmoney.text.replace(' ', '')) 
    right_team_money = soup.findAll("div",\
    class_ = "bet-button__content-main bet-currency bet-currency_RUB sys-stat-abs-2") # Количество поставленных денег на вторую команду
    for rmoney in right_team_money: # Наполнение списка
        right_team_money_all.append(rmoney.text.replace(' ', '')) 
    scores = soup.findAll("div",\
    class_ ="bet-match__additional-info-item bet-match__additional-info-item_result sys-time-fixed") # Найти счет матча
    winning_team_find = soup.select("div.bet-item")
    for wteam in winning_team_find:
        if "bet-winner-left" in wteam["class"]:
            winning_team_past.append("Первая команда") # Список победивших команд
        elif "bet-winner-right" in wteam["class"]:
            winning_team_past.append("Вторая команда")
        elif "bet-rescheduled" in wteam["class"]:
            winning_team_past.append("Матч перенесен")
        elif "bet-canceled" in wteam["class"]:
            winning_team_past.append("Матч отменен")
    # Проверка гипотезы
    for compare in range(len(left_team_money_all)): #перебираем элементы списка (в пределах длины первого списка)
        if ((int(left_team_money_all[compare]) > int(right_team_money_all[compare]))\
        and winning_team_past[compare] == "Первая команда")\
        or ((int(left_team_money_all[compare]) < int(right_team_money_all[compare]))\
        and winning_team_past[compare] == "Вторая команда"):
            hyps.append("Гипотеза верна")
        elif ((int(left_team_money_all[compare]) < int(right_team_money_all[compare]))\
        and winning_team_past[compare] == "Первая команда")\
        or ((int(left_team_money_all[compare]) > int(right_team_money_all[compare]))\
        and winning_team_past[compare] == "Вторая команда"):
            hyps.append("Гипотеза не верна")
        else:
            hyps.append("Гипотеза не проверялась")
    for ptf in range(len(left_teams)): 
        dictionary_past_results.append({"Left_team_title": left_teams[ptf].text,\
                                        "Moneybet_for_left_team": left_team_money[ptf].text.replace(' ', ''),\
                                        "Score": scores[ptf].text,\
                                        "Right_team_title": right_teams[ptf].text,\
                                        "Moneybet_for_right_team": right_team_money[ptf].text.replace(' ', ''),\
                                       "Winning_team_match_result": winning_team_past[ptf],\
                                       "Hypothesis": hyps[ptf],})

# Формула, которая считает процент верных и неверных гипотез через count ()      
model_success_percent = int(100 * hyps.count("Гипотеза верна") / (hyps.count("Гипотеза верна") + hyps.count("Гипотеза не верна")))
model_not_success_percent = int(100 * hyps.count("Гипотеза не верна") / (hyps.count("Гипотеза верна") + hyps.count("Гипотеза не верна")))

hyps_result = "Случаев выполнения гипотезы: {}, что составляет {}%; случаев, когда гипотеза была не выполнена {},что составляет {}%".format(hyps.count("Гипотеза верна"), model_success_percent, hyps.count("Гипотеза не верна"), model_not_success_percent)


# Создание csv файла с прошедшими матчами
with open('C:\Projects_Learnpython\BetProjectCS\past_CSmatchs.csv', 'w', newline='', encoding='utf-8') as ptcsv:
    ptfields = ["Left_team_title", "Moneybet_for_left_team",\
                "Score",\
                "Right_team_title", "Moneybet_for_right_team", "Winning_team_match_result",\
                "Hypothesis"]
    writer_past_matchs = csv.DictWriter(ptcsv, ptfields, delimiter=';')
    writer_past_matchs.writeheader()
    for row_past_matchs in dictionary_past_results:
        writer_past_matchs.writerow(row_past_matchs)

# Создание json файла с текущими матчами
with open ('C:\Projects_Learnpython\BetProjectCS\past_CSmatchs.json', 'w') as pts:
    json.dump("Результаты прошедших матчей: {}".format(dictionary_past_results), pts, ensure_ascii=False, sort_keys=True)
    json.dump("Результаты проверки гипотезы: {}".format(hyps_result), pts, ensure_ascii=False, sort_keys=True)

# Принты для проверки правильности работы программы
print(len(left_teams))
print(len(right_teams))    
print(len(left_team_money))
print(len(right_team_money))
print(dictionary_past_results)
print(len(left_team_money_all))
print(len(right_team_money_all))
print(len(winning_team_past))
print(len(dictionary_past_results))

29
29
29
29
[{'Left_team_title': ' 88 Keyd ', 'Moneybet_for_left_team': '193201', 'Score': '   16 : 12   ', 'Right_team_title': ' Bulldozer 139 ', 'Moneybet_for_right_team': '160552', 'Winning_team_match_result': 'Первая команда', 'Hypothesis': 'Гипотеза верна'}, {'Left_team_title': ' 86 EOX ', 'Moneybet_for_left_team': '683056', 'Score': '   16 : 14   ', 'Right_team_title': ' Santos 140 ', 'Moneybet_for_right_team': '308076', 'Winning_team_match_result': 'Первая команда', 'Hypothesis': 'Гипотеза верна'}, {'Left_team_title': ' 76 DETONA ', 'Moneybet_for_left_team': '520395', 'Score': '   16 : 10   ', 'Right_team_title': ' Isurus 73 ', 'Moneybet_for_right_team': '562852', 'Winning_team_match_result': 'Первая команда', 'Hypothesis': 'Гипотеза не верна'}, {'Left_team_title': ' 133 Red Canids ', 'Moneybet_for_left_team': '48361', 'Score': '   8 : 16   ', 'Right_team_title': ' Bulldozer 139 ', 'Moneybet_for_right_team': '19708', 'Winning_team_match_result': 'Вторая команда', 'Hypothesis': '

In [3]:
# Парсинг текущих событий

url_matchs = 'https://betscsgo.gg/'
driver.get(url_matchs)
time.sleep(15)
soup = BS(driver.page_source, 'html.parser')
left_teams = soup.findAll("div", class_="bet-team__name sys-t1name") # Название команды, написанной слева
right_teams = soup.findAll("div", class_="bet-team__name sys-t2name") # Название команды, написанной справа
left_team_money = soup.findAll("div",\
class_ = "bet-button__content-main bet-currency bet-currency_RUB sys-stat-abs-1") 
# Количество поставленных денег на первую команду
right_team_money = soup.findAll("div",\
class_ = "bet-button__content-main bet-currency bet-currency_RUB sys-stat-abs-2")
# Количество поставленных денег на вторую команду
koef_left = soup.findAll("div",\
class_ ="bet-button__content-more-item bet-button__content-more-item_coeff sys-stat-koef-1") # Коэффициент на первую команду
koef_right = soup.findAll("div",\
class_="bet-button__content-more-item bet-button__content-more-item_coeff sys-stat-koef-2") # Коэффициент на вторую команду
win_proc_left = soup.findAll("div",\
class_ ="bet-button__content-more-item bet-button__content-more-item_percent sys-stat-proc-1") # Найти счет матча
win_proc_right = soup.findAll("div",\
class_ ="bet-button__content-more-item bet-button__content-more-item_percent sys-stat-proc-2") 
time_before_match = soup.findAll("div",\
class_=["bet-match__additional-info-item bet-match__additional-info-item_timer sys-time-left",\
"bet-match__additional-info-item  sys-time-fixed"]) # Время до начала матчей

dictionary_present_results = [] # Создание списка словарей

for prf in range(len(left_teams)):
    dictionary_present_results.append({"Left_team_title": left_teams[prf].text,\
                                       "Left_team_coefficients": koef_left[prf].text,\
                                       "Moneybet_for_left_team": left_team_money[prf].text.replace(' ', ''),\
                                       "Left_team_win_percent": win_proc_left[prf].text,\
                                       "Time_before_match": time_before_match[prf].text,\
                                       "Right_team_title": right_teams[prf].text,\
                                       "Right_team_coefficients": koef_right[prf].text,\
                                       "Moneybet_for_right_team": right_team_money[prf].text.replace(' ', ''),\
                                       "Right_team_win_percent": win_proc_right[prf].text})

# Создание csv файла с текущими матчами 
with open ('C:\Projects_Learnpython\BetProjectCS\present_CSmatchs.csv', 'w', newline='', encoding='utf-8') as prcsv:
    prfields = ["Left_team_title", "Left_team_coefficients", "Moneybet_for_left_team", "Left_team_win_percent",\
              "Time_before_match",\
              "Right_team_title", "Right_team_coefficients", "Moneybet_for_right_team", "Right_team_win_percent"]
    writer_present_matchs = csv.DictWriter(prcsv, prfields, delimiter=';')
    writer_present_matchs.writeheader()
    for row_present_matchs in dictionary_present_results:
        writer_present_matchs.writerow(row_present_matchs)

# Создание json файла с текущими матчами 
with open('C:\Projects_Learnpython\BetProjectCS\present_CSmatchs.json', 'w') as prs:
    json.dump("Текущие результаты матчей: {}".format(dictionary_present_results), prs, ensure_ascii=False, sort_keys=True)

# Принты для проверки правильности работы программы
print(len(left_teams))
print(len(right_teams))
print(len(left_team_money))
print(len(right_team_money))
print(len(koef_left))
print(len(koef_right))
print(len(win_proc_left))
print(len(win_proc_right))
print(len(time_before_match))
print(dictionary_present_results)

14
14
14
14
14
14
14
14
14
[{'Left_team_title': ' 67 Ground Zero ', 'Left_team_coefficients': 'x3.42', 'Moneybet_for_left_team': '119798', 'Left_team_win_percent': '27%', 'Time_before_match': 'Скоро старт', 'Right_team_title': ' Order 35 ', 'Right_team_coefficients': 'x1.33', 'Moneybet_for_right_team': '322326', 'Right_team_win_percent': '73%'}, {'Left_team_title': ' 79 5power ', 'Left_team_coefficients': 'x1.96', 'Moneybet_for_left_team': '302001', 'Left_team_win_percent': '48%', 'Time_before_match': '10м', 'Right_team_title': ' AVANT 91 ', 'Right_team_coefficients': 'x1.85', 'Moneybet_for_right_team': '320978', 'Right_team_win_percent': '52%'}, {'Left_team_title': ' 18 Tyloo ', 'Left_team_coefficients': 'x1.21', 'Moneybet_for_left_team': '142358', 'Left_team_win_percent': '81%', 'Time_before_match': '10м', 'Right_team_title': ' FFAmix null ', 'Right_team_coefficients': 'x4.85', 'Moneybet_for_right_team': '33248', 'Right_team_win_percent': '19%'}, {'Left_team_title': ' 28 Grayhound ',

In [8]:
import pandas as pd
pastmatchs = pd.read_csv('C:\Projects_Learnpython\past_CSmatchs.csv', sep=';')
presentmatchs = pd.read_csv('C:\Projects_Learnpython\present_CSmatchs.csv', sep=';')

In [9]:
pastmatchs

,Left_team_title,Moneybet_for_left_team,Score,Right_team_title,Moneybet_for_right_team,Winning_team_match_result,Hypothesis
0,124 absolute mad lads,1954,2 : 1,Bushido Boyz null,2438,Первая команда,Гипотеза не верна
1,null mythic,4870,2 : 0,heX null,3579,Первая команда,Гипотеза верна
2,null Big Frames,6133,2 : 0,Prospects null,2874,Первая команда,Гипотеза верна
3,null Thunder Logic,10459,2 : 0,Just Swing null,8356,Первая команда,Гипотеза верна
4,4 Vitality,1082010,2 : 1,Heroic 16,347675,Первая команда,Гипотеза верна
5,null Fanta,3307,0 : 2,Bizarre null,2236,Вторая команда,Гипотеза не верна
6,77 DETONA,50145,отменен,Team Reapers 137,22679,Матч отменен,Гипотеза не проверялась
7,54 ALTERNATE aTTaX,12458,2 : 1,SJ Gaming 61,19816,Первая команда,Гипотеза не верна
8,63 expert,8395,1 : 2,AVANGAR 19,47417,Вторая команда,Гипотеза верна
9,65 Izako Boars,6638,1 : 2,Tricked 32,29065,Вторая команда,Гипотеза верна


In [10]:
presentmatchs

,Left_team_title,Left_team_coefficients,Moneybet_for_left_team,Left_team_win_percent,Time_before_match,Right_team_title,Right_team_coefficients,Moneybet_for_right_team,Right_team_win_percent
0,13 FURIA,x1.08,346637,92%,1ч 46м,Sharks 58,x10.82,31768,8%
1,128 Ignis,x1.69,110,57%,3ч 46м,Absolute null,x2.18,84,43%
2,7 NRG,x1.22,92534,80%,4ч 46м,INTZ 45,x4.66,22726,20%
3,null FATE,x1.04,1611,96%,5ч 46м,HEADSHOTBG null,x21.07,72,4%
4,48 Epsilon,x1.70,734,56%,6ч 46м,Unicorns of Love 70,x2.15,575,44%
5,12 CR4ZY,x1.50,35348,64%,7ч 46м,Fnatic 8,x2.63,19488,36%
6,54 ALTERNATE aTTaX,x36.30,8,2%,10ч 16м,AVANGAR 19,x1.02,314,98%
7,32 Tricked,x6.22,260,15%,10ч 16м,Nemiga 42,x1.16,1511,85%
8,49 HAVU,x1.00,1644,100%,10ч 16м,Fierce 87,NaN,0,0%
9,65 Izako Boars,x1.07,134,93%,10ч 16м,Defusekids null,x13.10,10,7%


In [11]:
#Проверка победы первой команды в прошлых матчах
lpastmatchs = pastmatchs[['Left_team_title', 'Right_team_title', 'Winning_team_match_result',\
            'Hypothesis']].loc[pastmatchs['Hypothesis'] == "Гипотеза верна"].\
          loc[pastmatchs['Winning_team_match_result'] == "Первая команда"]
lpastmatchs

,Left_team_title,Right_team_title,Winning_team_match_result,Hypothesis
1,null mythic,heX null,Первая команда,Гипотеза верна
2,null Big Frames,Prospects null,Первая команда,Гипотеза верна
3,null Thunder Logic,Just Swing null,Первая команда,Гипотеза верна
4,4 Vitality,Heroic 16,Первая команда,Гипотеза верна
12,100 Goliath,Sinister5 null,Первая команда,Гипотеза верна
14,70 Unicorns of Love,Copenhagen Flames 74,Первая команда,Гипотеза верна
17,19 AVANGAR,Defusekids null,Первая команда,Гипотеза верна
19,4 Vitality,Renegades 17,Первая команда,Гипотеза верна
22,61 SJ Gaming,Lyngby Vikings 84,Первая команда,Гипотеза верна
24,45 INTZ,Sharks 58,Первая команда,Гипотеза верна


In [5]:
#Проверка победы второй команды в прошлых матчах
rpastmatchs = pastmatchs[['Left_team_title', 'Right_team_title', 'Winning_team_match_result',\
            'Hypothesis']].loc[pastmatchs['Hypothesis'] == "Гипотеза верна"].\
          loc[pastmatchs['Winning_team_match_result'] == "Вторая команда"]
rpastmatchs

,Left_team_title,Right_team_title,Winning_team_match_result,Hypothesis
7,14 mibr,G2 9,Вторая команда,Гипотеза верна
15,36 Luminosity,FURIA 13,Вторая команда,Гипотеза верна
18,59 TeamOne,Singularity 47,Вторая команда,Гипотеза верна
20,135 Red Canids,paiN 85,Вторая команда,Гипотеза верна
27,33 NoChance,CR4ZY 12,Вторая команда,Гипотеза верна
28,74 Copenhagen Flames,Heretics 88,Вторая команда,Гипотеза верна
36,6 NRG,Sharks 60,Вторая команда,Гипотеза верна
44,null Dogs of War,Good Game PR null,Вторая команда,Гипотеза верна
47,77 DETONA,Isurus 73,Вторая команда,Гипотеза верна
49,null EOX,W7M 130,Вторая команда,Гипотеза верна


In [6]:
#Проверка первой команды в текущих матчах
lpresentmatchs = presentmatchs[['Left_team_title', 'Right_team_title',\
               'Moneybet_for_left_team', 'Moneybet_for_right_team']].\
             loc[presentmatchs['Moneybet_for_left_team'] > presentmatchs['Moneybet_for_right_team']]
lpresentmatchs

,Left_team_title,Right_team_title,Moneybet_for_left_team,Moneybet_for_right_team
4,2 Liquid,G2 9,189922,36846
5,43 INTZ,Sharks 60,2363,464
6,6 NRG,FURIA 13,4903,4079
7,61 SJ Gaming,Lyngby Vikings 84,5,0
8,12 CR4ZY,TBD null,182,11
9,101 Goliath,Sinister5 null,267,176


In [7]:
#Проверка второй команды в текущих матчах
rpresentmatchs = presentmatchs[['Left_team_title', 'Right_team_title',\
               'Moneybet_for_left_team', 'Moneybet_for_right_team']].\
             loc[presentmatchs['Moneybet_for_left_team'] < presentmatchs['Moneybet_for_right_team']]
rpresentmatchs

,Left_team_title,Right_team_title,Moneybet_for_left_team,Moneybet_for_right_team
0,33 NoChance,mousesports 5,123616,782678
1,16 Renegades,Ence 7,57730,161609
2,45 EnVy,mibr 14,31153,89092
3,138 Team Reapers,Isurus 73,10780,15788


In [8]:
lipresentmatchs = lpresentmatchs['Left_team_title']
lipresentmatchs

4         2 Liquid 
5          43 INTZ 
6            6 NRG 
7     61 SJ Gaming 
8         12 CR4ZY 
9      101 Goliath 
Name: Left_team_title, dtype: object

In [9]:
ripresentmatchs = rpresentmatchs['Right_team_title']
ripresentmatchs

0     mousesports 5 
1            Ence 7 
2           mibr 14 
3         Isurus 73 
Name: Right_team_title, dtype: object

In [10]:
top_teams = []
for lprmatch in list(lipresentmatchs):
    if (lprmatch in list(lpastmatchs['Left_team_title'])) and ((list(lpastmatchs['Hypothesis']).count('Гипотеза верна')) >=3)\
    or lprmatch in list(rpastmatchs['Right_team_title']) and ((list(lpastmatchs['Hypothesis']).count('Гипотеза верна')) >=3):
        print(lprmatch)
        top_teams.append(lprmatch)

 2 Liquid 
 43 INTZ 
 6 NRG 
 61 SJ Gaming 
 12 CR4ZY 
 101 Goliath 


In [23]:
top_teams_coloured = []
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
for rprmatch in list(ripresentmatchs):
    if (rprmatch in list(lpastmatchs['Left_team_title'])) and ((list(lpastmatchs['Hypothesis']).count('Гипотеза верна')) >=3)\
    or rprmatch in list(rpastmatchs['Right_team_title']) and ((list(lpastmatchs['Hypothesis']).count('Гипотеза верна')) >=3):
        print(rprmatch)
        top_teams.append(rprmatch)
for team in top_teams:
    top_teams_coloured.append(color.RED + color.BOLD + team + color.END)

print(top_teams_coloured)

 mousesports 5 
 Ence 7 
 mibr 14 
 Isurus 73 
['\x1b\x1b 2 Liquid \x1b', '\x1b\x1b 43 INTZ \x1b', '\x1b\x1b 6 NRG \x1b', '\x1b\x1b 61 SJ Gaming \x1b', '\x1b\x1b 12 CR4ZY \x1b', '\x1b\x1b 101 Goliath \x1b', '\x1b\x1b mousesports 5 \x1b', '\x1b\x1b Ence 7 \x1b', '\x1b\x1b mibr 14 \x1b', '\x1b\x1b Isurus 73 \x1b', '\x1b\x1b mousesports 5 \x1b', '\x1b\x1b Ence 7 \x1b', '\x1b\x1b mibr 14 \x1b', '\x1b\x1b Isurus 73 \x1b', '\x1b\x1b mousesports 5 \x1b', '\x1b\x1b Ence 7 \x1b', '\x1b\x1b mibr 14 \x1b', '\x1b\x1b Isurus 73 \x1b', '\x1b\x1b mousesports 5 \x1b', '\x1b\x1b Ence 7 \x1b', '\x1b\x1b mibr 14 \x1b', '\x1b\x1b Isurus 73 \x1b', '\x1b\x1b mousesports 5 \x1b', '\x1b\x1b Ence 7 \x1b', '\x1b\x1b mibr 14 \x1b', '\x1b\x1b Isurus 73 \x1b', '\x1b\x1b mousesports 5 \x1b', '\x1b\x1b Ence 7 \x1b', '\x1b\x1b mibr 14 \x1b', '\x1b\x1b Isurus 73 \x1b', '\x1b\x1b mousesports 5 \x1b', '\x1b\x1b Ence 7 \x1b', '\x1b\x1b mibr 14 \x1b', '\x1b\x1b Isurus 73 \x1b', '\x1b\x1b mousesports 5 \x1b', '\x1b\x1b Enc

In [73]:
list(lpastmatchs['Hypothesis']).count('Гипотеза верна') >=3

689

In [16]:
presentmatchs

,Left_team_title,Left_team_coefficients,Moneybet_for_left_team,Left_team_win_percent,Time_before_match,Right_team_title,Right_team_coefficients,Moneybet_for_right_team,Right_team_win_percent
0,33 NoChance,x6.70,123616,14%,Скоро старт,mousesports 5,x1.14,782678,14%
1,16 Renegades,x3.52,57730,26%,Скоро старт,Ence 7,x1.32,161609,26%
2,45 EnVy,x3.57,31153,26%,20м,mibr 14,x1.31,89092,26%
3,138 Team Reapers,x2.32,10780,41%,1ч 00м,Isurus 73,x1.61,15788,41%
4,2 Liquid,x1.17,189922,84%,2ч 40м,G2 9,x5.64,36846,84%
5,43 INTZ,x1.18,2363,84%,11ч 00м,Sharks 60,x5.59,464,84%
6,6 NRG,x1.75,4903,55%,14ч 00м,FURIA 13,x2.08,4079,55%
7,61 SJ Gaming,x1.00,5,100%,16ч 00м,Lyngby Vikings 84,NaN,0,100%
8,12 CR4ZY,x1.05,182,94%,17ч 00м,TBD null,x15.89,11,94%
9,101 Goliath,x1.59,267,60%,21ч 30м,Sinister5 null,x2.36,176,60%
